In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vnstock as vn

def ema(data, window):
    return data.ewm(span=window, adjust=False).mean()

def generate_signals(data):
    signals = pd.DataFrame(index=data.index)
    signals['EMA5'] = ema(data, 5)
    signals['EMA8'] = ema(data, 8)
    signals['EMA13'] = ema(data, 13)
    
    # Buy Signal: EMA10 crosses up 2 lines (EMA8 and EMA13)
    signals['Buy_Signal'] = np.where((signals['EMA5'] > signals['EMA8']) &
                                     (signals['EMA5'] > signals['EMA13']) &
                                     (signals['EMA5'].shift(3) < signals['EMA8'].shift(3)) &
                                     (signals['EMA5'].shift(3) < signals['EMA13'].shift(3)), 1, 0)
    
    # Sell Signal: EMA10 crosses down 2 lines (EMA8 and EMA13)
    signals['Sell_Signal'] = np.where((signals['EMA5'] < signals['EMA8']) &
                                      (signals['EMA5'] < signals['EMA13']) &
                                      (signals['EMA5'].shift(3) > signals['EMA8'].shift(3)) &
                                      (signals['EMA5'].shift(3) > signals['EMA13'].shift(3)), 1, 0)
    
    return signals

def stock_trading_performance(data, signals):
    position = 0  # 0 indicates no position, 1 indicates long position
    entry_price = 0  # Entry price for the position
    trading_log = []  # List to store trading log
    for index, signal in signals.iterrows():
        close_price = data.loc[index, 'close']
        if signal['Buy_Signal'] == 1 and position != 1:
            # Buy Signal
            if position == 0:  # No position
                entry_price = close_price
            position = 1
        elif signal['Sell_Signal'] == 1 and position != 0:
            # Sell Signal (Closing long position)
            if position == 1:  # Long position
                profit = (close_price - entry_price) / entry_price
                trading_log.append(profit)
                position = 0  # No position after closing
    # Calculate performance returns
    perf_index_returns = np.array(trading_log)
    
    # Calculate cumulative return
    cumulative_return = np.prod(np.array(trading_log) + 1) - 1

    # Calculate volatility
    volatility = np.std(perf_index_returns)
    return cumulative_return, volatility

def portfolio_trading_performance(symbols, start_date, end_date, resolution, weights):
    cumulative_returns = []
    volatilities = []
    for i, symbol in enumerate(symbols):
        # Load data for each stock into a DataFrame
        data = vn.stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution=resolution, type="stock", beautify=True, decor=False, source='DNSE')
        # Generate buy and sell signals for each stock
        signals = generate_signals(data['close'])
        # Calculate trading performance for each stock
        cumulative_return, volatility = stock_trading_performance(data, signals)
        # Adjust cumulative return with weight
        cumulative_return *= weights[i]
        cumulative_returns.append(cumulative_return)
        volatilities.append(volatility)
    # Calculate weighted average portfolio performance
    portfolio_cumulative_return = np.sum(cumulative_returns)
    portfolio_volatility = np.mean(volatilities)  # Assuming equal weights for volatility calculation
    return portfolio_cumulative_return, portfolio_volatility

# Define symbols, start_date, end_date, resolution, and weights for the portfolio
symbols = ["PHR", "QNS", "NTL", "PTB", "STK"]
start_date = "2018-04-01"
end_date = "2019-04-02"
resolution = "1D"
weights = [0.3513, 0.05, 0.3530, 0.1957, 0.05]

# Calculate trading performance for the portfolio with weights
portfolio_cumulative_return, portfolio_volatility = portfolio_trading_performance(symbols, start_date, end_date, resolution, weights)

# Print portfolio performance with weights
portfolio_cumulative_return_percentage = portfolio_cumulative_return * 100
portfolio_volatility_percentage = portfolio_volatility * 100
print("Portfolio Cumulative Return with Weights:", "{:.2f}%".format(portfolio_cumulative_return_percentage))
print("Portfolio Volatility with Weights:", "{:.2f}%".format(portfolio_volatility_percentage))


Portfolio Cumulative Return with Weights: 22.08%
Portfolio Volatility with Weights: 12.32%


In [10]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

def plot_trading_performance(symbols, start_date, end_date, resolution, weights):
    fig = make_subplots(rows=len(symbols), cols=1, shared_xaxes=True, subplot_titles=symbols)

    for i, symbol in enumerate(symbols):
        # Load data for each stock into a DataFrame
        data = vn.stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution=resolution, type="stock", beautify=True, decor=False, source='DNSE')
        # Generate buy and sell signals for each stock
        signals = generate_signals(data['close'])
        # Calculate trading performance for each stock
        cumulative_return, volatility = stock_trading_performance(data, signals)
        # Plot cumulative return
        fig.add_trace(go.Scatter(x=data.index, y=np.cumprod(1 + signals['Buy_Signal'] * (data['close'].pct_change())), mode='lines', name=f'{symbol} Cumulative Return'), row=i+1, col=1)
        
    fig.update_layout(height=600*len(symbols), title_text="Trading Performance Dashboard")
    fig.update_yaxes(title_text="Cumulative Return", row=len(symbols)//2+1, col=1)
    fig.update_xaxes(title_text="Date", row=len(symbols), col=1)
    fig.show()

# Call the function to plot trading performance
plot_trading_performance(symbols, start_date, end_date, resolution, weights)


In [11]:
def plot_trading_performance_with_signals(symbol, start_date, end_date, resolution):
    # Load data for the stock into a DataFrame
    data = vn.stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution=resolution, type="stock", beautify=True, decor=False, source='DNSE')
    # Generate buy and sell signals for the stock
    signals = generate_signals(data['close'])

    # Create a Plotly figure
    fig = go.Figure()

    # Plot historical data
    fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close Price'))

    # Plot buy signals
    buy_signals = signals[signals['Buy_Signal'] == 1]
    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['EMA5'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

    # Plot sell signals
    sell_signals = signals[signals['Sell_Signal'] == 1]
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['EMA5'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

    # Update layout
    fig.update_layout(title=f"Stock Data and Buy/Sell Signals for {symbol}", xaxis_title="Date", yaxis_title="Price")
    fig.show()

# Call the function to plot trading performance with signals for a specific symbol
plot_trading_performance_with_signals("PHR", start_date, end_date, resolution)


In [12]:
def print_trading_signals(symbol, start_date, end_date, resolution):
    # Load data for the stock into a DataFrame
    data = vn.stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution=resolution, type="stock", beautify=True, decor=False, source='DNSE')
    # Generate buy and sell signals for the stock
    signals = generate_signals(data['close'])

    # Filter buy and sell signals
    buy_signals = signals[signals['Buy_Signal'] == 1]
    sell_signals = signals[signals['Sell_Signal'] == 1]

    print("Buy Signals:")
    print(buy_signals.index)

    print("\nSell Signals:")
    print(sell_signals.index)

# Call the function to print trading signals for a specific symbol
print_trading_signals("NT2", start_date, end_date, resolution)


Buy Signals:
Index([  7,   8,  32,  60,  61,  62,  84,  85, 100, 122, 123, 139, 140, 141,
       170, 171, 172, 209, 249, 250],
      dtype='int64')

Sell Signals:
Index([  4,   5,  15,  16,  17,  35,  72,  77,  89,  90,  91, 119, 120, 129,
       130, 150, 151, 191, 192, 193, 240, 241],
      dtype='int64')
